In [1]:
import cv2 as cv
import mediapipe as mp

In [2]:
def landmark_bbox(hand_landmarks, height, width) -> float :
    x_min, y_min = width, height
    x_max, y_max = 0, 0

    for landmark in hand_landmarks.landmark:
        x, y = landmark.x * width, landmark.y * height

        x_min = min(x_min, x)
        y_min = min(y_min, y)
        x_max = max(x_max, x)
        y_max = max(y_max, y)
    return x_min, y_min, x_max, y_max

In [3]:
def normalized_landmarks(hand_landmarks, bbox, height, width) -> float:
    x_min, y_min, x_max, y_max = bbox
    bbox_W, bbox_H = (x_max - x_min), (y_max - y_min)
    normalized_landmarks = []
    for landmark in hand_landmarks.landmark:
        x = (landmark.x * width - x_min) / bbox_W
        y = (landmark.y * height - y_min) / bbox_H
        normalized_landmarks.append((x, y))

    return normalized_landmarks

In [4]:
PATH = 'data/videos/hand_shake.mp4'

In [5]:
video = cv.VideoCapture(PATH)

hands = mp.solutions.hands.Hands()
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

Trail = []
while(video.isOpened()):
    ret, frame = video.read()
    if ret == True:
    
        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)
        H, W, _ = frame.shape
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                height, width, _ = frame.shape
                cx, cy = int(index_finger.x * width), int(index_finger.y * height)
                Trail.append((cx, cy))
                bbox = landmark_bbox(hand_landmarks, H, W)
                posotion = normalized_landmarks(hand_landmarks, bbox, H, W)
                # print(posotion)
                
                # mp_drawing.draw_landmarks(
                # frame,
                # hand_landmarks,
                # mp_hands.HAND_CONNECTIONS,  
                # mp_drawing_styles.get_default_hand_landmarks_style(),
                # mp_drawing_styles.get_default_hand_connections_style())
                
                
        for i in Trail:
            cv.circle(frame, i, 10, (0, 255, 0), cv.FILLED)
        cv.imshow('frame', frame)
        
        if cv.waitKey(25) & 0xFF == ord('q'):
              break
    else: 
        break
video.release()
cv.destroyAllWindows()